This is the file were using to test the llama api.

Create Api request object: 

We define the conent a

In [9]:
import pandas as pd
import numpy as np
import seaborn as sn
import json
import matplotlib.pyplot as plt
from collections import Counter
from llamaapi import LlamaAPI
from dotenv import load_dotenv
import os 
import requests
from llama_index.core.llms import LLM
from llama_index.llms.openai import OpenAI

Creating the LLAMA object to start using the API

In [10]:
load_dotenv()
# Create a new LlamaAPI object
LLAMA_API_KEY = os.getenv("LLAMA_API_KEY")
LLAMA_API_URL = os.getenv("LLAMA_API_URL") 
llama = LlamaAPI(LLAMA_API_KEY)

We define the content as: What is the sector and industry of this job

We give the API the following infomration: job description, responsibilities, and skills 

In [11]:
api_request_json = {
  "model": "llama3.1-70b",
  "messages": [
    {"role": "user", "content": "What is the sector and industry for the following job: Skills: {skills}, Responsibilities: {responsibilities}, Description: {description}?"}, 
  ],
  "functions": [
    {
        "name": "get_sector_industry",
        "description": 
            "Get the sector and industry from this given job information such as skills, responsibilites, and description",
        "parameters": {
            "type": "object",
        "properties": { 
            # "job_description": {
            #     "type": "string",
            #     "description": "The job description for the role"
            # },
            # "responsibilites": {
            #     "type": "string",
            #     "description": "The job responsibilites for the role"
            # },
            # "skills": {
            #     "type": "string",
            #     "description": "The required skills listed for the role"
            # },
            "company": {
                "type": "string",
                "description": "The name of the company"
            },
        },
        "required": ["job description", "responsibilites", "skills"] 
        }
    }
  ],
}


Prepare json object for the api request

In [12]:
df = pd.read_csv("job_descriptions.csv")

# selected_columns = df[['skills', 'Responsibilities', 'Job Description', 'Company']]
df = pd.DataFrame(df['Company']); 
df = df.head(50)
print(df)

                                  Company
0                       Icahn Enterprises
1            PNC Financial Services Group
2        United Services Automobile Assn.
3                                    Hess
4                            Cairn Energy
5   Adani Ports and Special Economic Zone
6                                   FedEx
7                            Ryder System
8           Zee Entertainment Enterprises
9                                     CSX
10                             McDonald's
11                            TPG Telecom
12                        ThyssenKrupp AG
13                       EnLink Midstream
14                    NGL Energy Partners
15                               Bayer AG
16                                  Deere
17                 Mondelez International
18                                     3M
19                                    KKR
20                    NGL Energy Partners
21                                EasyJet
22                         Otis Wo

In [13]:
def prepare_json(df):
    # Convert DataFrame into JSON-ready format
    job_listings = []
    for index, row in df.iterrows():
        listing = {
            # "skills": row['skills'],
            # "responsibilities": row['Responsibilities'],
            # "description": row['Job Description'],
            "company": row['Company']
        }
        job_listings.append(listing)
    
    # Return the JSON structure that will be sent to the API
    json_job = {
        "job_listings": job_listings  # List of jobs to be analyzed by Llama API
    }
    return json_job

# Prepare the JSON object
json_job = prepare_json(df)

# To see the structure of the JSON
print(json.dumps(json_job, indent=4))

{
    "job_listings": [
        {
            "company": "Icahn Enterprises"
        },
        {
            "company": "PNC Financial Services Group"
        },
        {
            "company": "United Services Automobile Assn."
        },
        {
            "company": "Hess"
        },
        {
            "company": "Cairn Energy"
        },
        {
            "company": "Adani Ports and Special Economic Zone"
        },
        {
            "company": "FedEx"
        },
        {
            "company": "Ryder System"
        },
        {
            "company": "Zee Entertainment Enterprises"
        },
        {
            "company": "CSX"
        },
        {
            "company": "McDonald's"
        },
        {
            "company": "TPG Telecom"
        },
        {
            "company": "ThyssenKrupp AG"
        },
        {
            "company": "EnLink Midstream"
        },
        {
            "company": "NGL Energy Partners"
        },
        {
           

In [19]:
print(f"API URL: {LLAMA_API_URL}")

def get_sector_industry(json_job):
    headers = {
        'Authorization': f'Bearer {LLAMA_API_KEY}', 
        #'Content-Type': 'application/json'
    }
    
    try:
        # Send POST request to Llama API with the job data
        response = requests.post(LLAMA_API_URL, headers=headers, data=json.dumps(json_job))
        
        # Check if the response is successful
        if response.status_code == 200:
            # Parse the JSON response
            results = response.json()
            
            # Extract sector and industry information
            sectors = [res['Sector'] for res in results['job_listings']]
            industries = [res['Industry'] for res in results['job_listings']]

            print(sectors)
            
            return sectors, industries
        else:
            print(f"Error: {response.status_code} - {response.text}")
            return None, None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None, None

# Using the function to get sector and industry for the job listings
sectors, industries = get_sector_industry(json_job)

# Adding the sector and industry back into the DataFrame
if sectors and industries:
    df['Sector'] = sectors
    df['Industry'] = industries

# Display the DataFrame with new columns
print(df)
# caroline says to not use a specific external url/database to query for info -- this is what the email and weather functions used but our case is different
# Need to create function to get skills, responsibilites, and description from job posting

API URL: https://llama.ai/api/v1/predict
Error: 404 - <html>
<head><title>404 Not Found</title></head>
<body bgcolor="white">
<center><h1>404 Not Found</h1></center>
<hr><center>openresty</center>
</body>
</html>

                                  Company
0                       Icahn Enterprises
1            PNC Financial Services Group
2        United Services Automobile Assn.
3                                    Hess
4                            Cairn Energy
5   Adani Ports and Special Economic Zone
6                                   FedEx
7                            Ryder System
8           Zee Entertainment Enterprises
9                                     CSX
10                             McDonald's
11                            TPG Telecom
12                        ThyssenKrupp AG
13                       EnLink Midstream
14                    NGL Energy Partners
15                               Bayer AG
16                                  Deere
17                 Mondelez In